In [36]:
import os
import pandas as pd
import tensorflow as tf
# import tensorflow_ranking as tfr
import numpy as np
from joblib import Parallel, delayed

In [38]:
data_dir = "/kaggle/working"
race_result_csv     = os.path.join(data_dir, "prepared.csv")
df_race_result   = pd.read_csv(race_result_csv)

print(len(df_race_result))
df_race_result.head()

1560664


,レースID,レース日付,距離(m),着順,馬名,馬齢,斤量,単勝,人気,場体重増減,...,馬番_15,馬番_16,馬番_17,馬番_18,馬番_19,馬番_20,馬番_21,馬番_22,馬番_23,馬番_24
0,202102010902,2021-07-31,0.414163,7.0,バンブトンローズ,0.171856,0.347738,0.481574,0.709403,0.50,...,0,0,0,0,0,0,0,0,0,0
1,202102010911,2021-07-31,0.541041,4.0,ヒシヴィクトリー,0.506159,0.393765,0.367208,0.548869,0.25,...,0,0,0,0,0,0,0,0,0,0
2,202102010910,2021-07-31,0.142285,12.0,エムオーシャトル,0.171856,0.253119,0.364726,0.637217,0.75,...,1,0,0,0,0,0,0,0,0,0
3,202102010910,2021-07-31,0.142285,13.0,マリノディアナ,0.506159,0.393765,0.683876,0.823304,0.75,...,0,0,0,0,0,0,0,0,0,0
4,202102010910,2021-07-31,0.142285,14.0,ラキ,0.305157,0.393765,0.564725,0.741094,-1.00,...,0,0,0,0,0,0,0,0,0,0


４次元テンソルとして学習データをまとめる。\
学習データ単位：１レース（出走馬 × 直近5レースの特徴量）

データセット数とForループの多重構造がかなり重いので、分散メモリ型の並列計算を行う。データフレームは各プロセスで同じものを定義するため、データフレームについてプロセス数倍のメモリが必要になることに注意。

In [ ]:
def process_race(race_id, df_race_result):
    # 各race_idに対する処理
    race_df = df_race_result[df_race_result['レースID'] == race_id].copy()
    horse_names = race_df['馬名'].unique()
    # 各レースのレース日付を、最初の馬のものとして取得
    race_date = race_df[race_df['馬名'] == horse_names[0]]['レース日付'].iloc[0]
    level2 = []  # 各馬についてのリスト

    for horse_name in horse_names:
        # 馬名におけるデータを抽出
        horse_df = df_race_result[df_race_result['馬名'] == horse_name].copy()
        horse_df.reset_index(drop=True, inplace=True)
        # 指定したレース日のインデックスを取得
        unique_id = horse_df.loc[horse_df['レース日付'] == race_date].index[0]
        # 不要な列の削除
        horse_df.drop(['レースID', 'レース日付', '馬名', '着順'], axis=1, inplace=True)
        level3 = []  # 直近5レースのデータを入れるリスト

        for k in range(5):
            if unique_id + k <= len(horse_df) - 1:
                record = horse_df.loc[unique_id + k]
                # bool型の場合は int に変換、list化
                record_as_list = record.apply(lambda x: int(x) if isinstance(x, bool) else x).tolist()
                level3.append(record_as_list)
            else:
                level3.append(0)
        level2.append(level3)
    return level2

# メイン処理
race_ids = df_race_result['レースID'].unique()

# joblib を用いて各レースIDの処理を並列化（プロセスベース、分散メモリ並列）
tensor_list = Parallel(n_jobs=-1, backend="loky")(
    delayed(process_race)(race_id, df_race_result) for race_id in race_ids
)


In [ ]:
# race_ids = df_race_result['レースID'].unique()
# tensor_list = []

# horse_df = df_race_result
# for i, race_id in enumerate(race_ids):
#     race_df = df_race_result[df_race_result['レースID'] == race_id].copy()
#     horse_names = race_df['馬名'].unique()
#     race_date = race_df[race_df['馬名'] == horse_names[0]]['レース日付'].iloc[0]
#     level2 = []
#     for j, horse_name in enumerate(horse_names):
#         horse_df = df_race_result[df_race_result['馬名'] == horse_name].copy()
#         horse_df.reset_index(drop=True, inplace=True)
#         unique_id = horse_df.loc[horse_df['レース日付'] == race_date].index[0]
#         horse_df.drop(['レースID', 'レース日付', '馬名', '着順'], axis=1, inplace=True)
#         # print(unique_id)
#         level3 = []
#         for k in range(5):
#             if unique_id + k <= len(horse_df) - 1:
#                 record = horse_df.loc[k]
#                 record_as_list = record.apply(lambda x: int(x) if isinstance(x, bool) else x).tolist()
#                 # print(record_as_list)
#                 # print(record)
#                 level3.append(record_as_list)
#             else:
#                 level3.append(0)
#             # break
#         level2.append(level3)
#         # break
#     tensor_list.append(level2)
#     # break

# horse_df.head()

In [ ]:
print(len(tensor_list), len(tensor_list[0]), len(tensor_list[0][0]), len(tensor_list[0][0][0]))
print(len(df_race_result))

In [ ]:
pop_list = []
for i in range(len(tensor_list)):
    for j in range(len(tensor_list[i])):
        if tensor_list[i][j][4] == 0:
            pop_list.append(i)
            break

filtered_list = [item for idx, item in enumerate(tensor_list) if idx not in pop_list]
print(len(pop_list), len(filtered_list), len(filtered_list[0]), len(filtered_list[0][0]), len(filtered_list[0][0][0]))